In [ ]:
from tkinter.filedialog import askopenfilename
import pandas as pd
import tkinter
import os
import numpy as np

tk_window = tkinter.Tk()
cwd = os.getcwd()

tr_data = pd.read_csv(askopenfilename(title='Choose your training data'))

tr_ans = pd.read_csv(askopenfilename(title='Choose your training answer'))

tr_ans = tr_ans.iloc[:, 0]

ts_data = pd.read_csv(askopenfilename(title='Choose your test data'))

tk_window.destroy()


In [ ]:
from sklearn.ensemble import RandomForestClassifier 
from sklearn.ensemble import GradientBoostingClassifier

tr_data.drop(['state', 'area_code', 'phone_number'], axis=1, inplace=True)
ts_data.drop(['state', 'area_code', 'phone_number'], axis=1, inplace=True)
from sklearn.preprocessing import LabelBinarizer
lb1 = LabelBinarizer()
lb2 = LabelBinarizer()
lb1.fit(tr_data['intl_plan'])
lb2.fit(tr_data['voice_mail_plan'])
tr_data['intl_plan'] = lb1.transform(tr_data['intl_plan'])
tr_data['voice_mail_plan'] = lb2.transform(tr_data['voice_mail_plan'])
ts_data['intl_plan'] = lb1.transform(ts_data['intl_plan'])
ts_data['voice_mail_plan'] = lb2.transform(ts_data['voice_mail_plan'])

# orange model
y_pred_list = []
model = RandomForestClassifier(n_estimators=2000, random_state=0, n_jobs=-1)
model = model.fit(tr_data, tr_ans)
y_pred = model.predict(ts_data)
y_pred_list.append(y_pred)                

model = GradientBoostingClassifier(n_estimators = 1000, learning_rate = 0.13, max_depth = 6, random_state=0)
model = model.fit(tr_data, tr_ans)
y_pred = model.predict(ts_data)
y_pred_list.append(y_pred)
if ts_data.shape[0] <= 1010: # 0.202 이하일시
    model = GradientBoostingClassifier(n_estimators = 1000, learning_rate = 0.131, max_depth = 6, random_state=0)
    model = model.fit(tr_data, tr_ans)
    y_pred = model.predict(ts_data)
    y_pred_list.append(y_pred)
else:
    model = GradientBoostingClassifier(n_estimators = 1000, learning_rate = 0.13, max_depth = 7, random_state=0)
    model = model.fit(tr_data, tr_ans)
    y_pred = model.predict(ts_data)
    y_pred_list.append(y_pred)  

d = {'0': y_pred_list[0], '1': y_pred_list[1], '2':y_pred_list[2]}
pred_dfs = pd.DataFrame(d)   
y_pred = pred_dfs.mode(axis=1)[0].values

pred_df = pd.DataFrame(y_pred)

pred_df.to_csv("output.csv", mode='w')

In [ ]:
def accuracy(real, predict):
    return sum(real == predict) / float(real.shape[0])

tk_window = tkinter.Tk()
cwd = os.getcwd()
ts_ans = pd.read_csv(askopenfilename(title='Choose your test answer'))
ts_ans = ts_ans.iloc[:, 0]
tk_window.destroy()

print(accuracy(ts_ans, y_pred))